In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import regex as re

In [3]:
#path for the file
file_path="pizza.txt"

#reading the file
with open (file_path,"r") as file:
    text=file.read()

    #spliting the data input according to the regex provided
    data=[sentence.strip() for sentence in re.split(r'(?<=[.!?]\s+)',text) if sentence.strip()]
# print(data)


In [4]:
#making an instance of the tokenizer
# tokenizer is used to map each unique word into unique integer
tokenizer=Tokenizer()

#scans through all the texts and keeps track of the frequency of each word in the texts
tokenizer.fit_on_texts(data)
# +1 is to act as a place holder for out-of-vocabulary
total_words=len(tokenizer.word_index)+1
# print(total_words)

In [5]:
input_sequences=[]
#making n-gram list for input text
for line in data:
    #converting the line by line into tokens
    token_list=tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        #making n-gram sequence
        #eg if the line is [1,2,3,4,5] then the n-gram sequence will be [1,2],[1,2,3],[1,2,3,4],[1,2,3,4,5]
        n_gram_sequence=token_list[:i+1]
        input_sequences.append(n_gram_sequence)
# print(input_sequences)

In [6]:
max_sequence_len=max([len(seq) for seq in input_sequences],default=0)
# padding the sequence to take the every input to the same length
input_sequences=np.array(pad_sequences(input_sequences,maxlen=max_sequence_len,padding="pre"))

# choosing X,y
# X = choosing all the list inside the list except the last item of each list
# y = choosing all the last item of the each list
# by this we can predict the next word(Integer) using the sequence of words (Integers)
X,y=input_sequences[:,:-1],input_sequences[:,-1]

In [7]:
#one hot encoding the y
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

In [8]:
model=Sequential()
#adding a embedding layer to the sequendial model
#translates high-dimensional data (like one-hot encoded vectors) into a lower-dimensional space.
#here we are converting the total words into 10 dimensional vector
model.add(Embedding(total_words,10,input_length=max_sequence_len-1))
#adding Long short term memory(RNN) to the model with 128 neurons
model.add(LSTM(128))
#adding dense(Fully connected layer to the model)
model.add(Dense(total_words,activation="softmax"))
#
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [9]:

# Train the model
model.fit(X, y, epochs=500, verbose=1)

Epoch 1/500
51/51 [==============================] - 4s 20ms/step - loss: 6.2232 - accuracy: 0.0516
Epoch 2/500
51/51 [==============================] - 1s 21ms/step - loss: 5.7978 - accuracy: 0.0547
Epoch 3/500
51/51 [==============================] - 1s 22ms/step - loss: 5.7285 - accuracy: 0.0559
Epoch 4/500
51/51 [==============================] - 1s 20ms/step - loss: 5.6987 - accuracy: 0.0553
Epoch 5/500
51/51 [==============================] - 1s 20ms/step - loss: 5.6654 - accuracy: 0.0565
Epoch 6/500
51/51 [==============================] - 1s 19ms/step - loss: 5.6170 - accuracy: 0.0498
Epoch 7/500
51/51 [==============================] - 1s 19ms/step - loss: 5.5658 - accuracy: 0.0528
Epoch 8/500
51/51 [==============================] - 1s 24ms/step - loss: 5.5161 - accuracy: 0.0565
Epoch 9/500
51/51 [==============================] - 1s 19ms/step - loss: 5.4621 - accuracy: 0.0670
Epoch 10/500
51/51 [==============================] - 1s 19ms/step - loss: 5.4068 - accuracy: 0.0627

In [13]:

# Generate next word predictions
seed_text = "Pizza have different "
next_words = 10
 
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences(
        [token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted_probs = model.predict(token_list)
    predicted_word = tokenizer.index_word[np.argmax(predicted_probs)]
    seed_text += " " + predicted_word
 
print("Next predicted words:", seed_text)

1/1 [==============================] - 0s 32ms/step
Next predicted words: Pizza have different  become a significant role in shaping the future of pizza
